# Herramientas para procesamiento de texto (parte 1)

In [1]:
# Solo si se usa Google Colab
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ejemplo_features1').getOrCreate()

In [3]:
df = spark.createDataFrame(
    [(0, 'Hola esta prueba es en el año 2021'),
     (1, 'Spark permite clasificación, regresión, etc.'),
     (2, 'Los,modelos,de,regresión,usan,datos,numéricos')
    ],
    ['ID', 'oracion']
    )

df.show(truncate=False)

+---+---------------------------------------------+
|ID |oracion                                      |
+---+---------------------------------------------+
|0  |Hola esta prueba es en el año 2021           |
|1  |Spark permite clasificación, regresión, etc. |
|2  |Los,modelos,de,regresión,usan,datos,numéricos|
+---+---------------------------------------------+



### Tokenizer

La "tokenización" (tokenization) es el proceso de tomar texto como entrada y dividirlo en términos individuales, usualmente palabras. 

In [4]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType

In [5]:
# Tokenizer por defecto (separa según espacios)
tokenizer = Tokenizer(inputCol='oracion', outputCol='palabras')

# Aplicar la tokenización a los datos
df2 = tokenizer.transform(df)

df2.show(truncate=False)

+---+---------------------------------------------+--------------------------------------------------+
|ID |oracion                                      |palabras                                          |
+---+---------------------------------------------+--------------------------------------------------+
|0  |Hola esta prueba es en el año 2021           |[hola, esta, prueba, es, en, el, año, 2021]       |
|1  |Spark permite clasificación, regresión, etc. |[spark, permite, clasificación,, regresión,, etc.]|
|2  |Los,modelos,de,regresión,usan,datos,numéricos|[los,modelos,de,regresión,usan,datos,numéricos]   |
+---+---------------------------------------------+--------------------------------------------------+



In [6]:
# Función definida por el usuario (UDF) para contar el número de palabras (tokens)
#     formato:  (funcion, tipo_retornado)
countTokens = udf(lambda x: len(x), IntegerType())

# Crear una nueva columna "ntokens" que muestra el número de tokens (palabras)
df2.withColumn('ntokens', countTokens(col('palabras'))).show()

+---+--------------------+--------------------+-------+
| ID|             oracion|            palabras|ntokens|
+---+--------------------+--------------------+-------+
|  0|Hola esta prueba ...|[hola, esta, prue...|      8|
|  1|Spark permite cla...|[spark, permite, ...|      5|
|  2|Los,modelos,de,re...|[los,modelos,de,r...|      1|
+---+--------------------+--------------------+-------+



In [7]:
# Tokenizer usando expresiones regulares (útil para el "español")
regexTokenizer = RegexTokenizer(inputCol='oracion', outputCol='palabras', pattern=r'[^\p{L}]+')  # '\\W'

df3 = regexTokenizer.transform(df)
df3.show(truncate=False)

+---+---------------------------------------------+-----------------------------------------------------+
|ID |oracion                                      |palabras                                             |
+---+---------------------------------------------+-----------------------------------------------------+
|0  |Hola esta prueba es en el año 2021           |[hola, esta, prueba, es, en, el, año]                |
|1  |Spark permite clasificación, regresión, etc. |[spark, permite, clasificación, regresión, etc]      |
|2  |Los,modelos,de,regresión,usan,datos,numéricos|[los, modelos, de, regresión, usan, datos, numéricos]|
+---+---------------------------------------------+-----------------------------------------------------+



In [8]:
df3.withColumn('ntokens', countTokens(col('palabras'))).show(truncate=False)

+---+---------------------------------------------+-----------------------------------------------------+-------+
|ID |oracion                                      |palabras                                             |ntokens|
+---+---------------------------------------------+-----------------------------------------------------+-------+
|0  |Hola esta prueba es en el año 2021           |[hola, esta, prueba, es, en, el, año]                |7      |
|1  |Spark permite clasificación, regresión, etc. |[spark, permite, clasificación, regresión, etc]      |5      |
|2  |Los,modelos,de,regresión,usan,datos,numéricos|[los, modelos, de, regresión, usan, datos, numéricos]|7      |
+---+---------------------------------------------+-----------------------------------------------------+-------+



### Stop Words

Son palabras que deben ser excluidas de la entrada, usualmente porque aparecen con mucha frecuencia y no contienen mucho significado ni aportan mucha información.

In [9]:
# Eliminar palabras comunes
from pyspark.ml.feature import StopWordsRemover

# Remover palabras comunes del idioma español
remover = StopWordsRemover(inputCol='palabras', outputCol='palabras filtradas', 
                           stopWords=StopWordsRemover.loadDefaultStopWords('spanish'))

df4 = remover.transform(df3)

df4.show(truncate=False)

+---+---------------------------------------------+-----------------------------------------------------+-----------------------------------------------+
|ID |oracion                                      |palabras                                             |palabras filtradas                             |
+---+---------------------------------------------+-----------------------------------------------------+-----------------------------------------------+
|0  |Hola esta prueba es en el año 2021           |[hola, esta, prueba, es, en, el, año]                |[hola, prueba, año]                            |
|1  |Spark permite clasificación, regresión, etc. |[spark, permite, clasificación, regresión, etc]      |[spark, permite, clasificación, regresión, etc]|
|2  |Los,modelos,de,regresión,usan,datos,numéricos|[los, modelos, de, regresión, usan, datos, numéricos]|[modelos, regresión, usan, datos, numéricos]   |
+---+---------------------------------------------+-------------------------

### N-gramas

Un n-grama es una secuencia de $n$ tokens (típicamente palabras) para algún número entero $n$. 

In [10]:
# n-gramas
from pyspark.ml.feature import NGram

ngram = NGram(n=2, inputCol='palabras', outputCol='ngrama')

ngram.transform(df3).show(truncate=False)

+---+---------------------------------------------+-----------------------------------------------------+------------------------------------------------------------------------------------+
|ID |oracion                                      |palabras                                             |ngrama                                                                              |
+---+---------------------------------------------+-----------------------------------------------------+------------------------------------------------------------------------------------+
|0  |Hola esta prueba es en el año 2021           |[hola, esta, prueba, es, en, el, año]                |[hola esta, esta prueba, prueba es, es en, en el, el año]                           |
|1  |Spark permite clasificación, regresión, etc. |[spark, permite, clasificación, regresión, etc]      |[spark permite, permite clasificación, clasificación regresión, regresión etc]      |
|2  |Los,modelos,de,regresión,usan,datos,numé

In [11]:
ngram = NGram(n=3, inputCol='palabras', outputCol='ngrama')

ngram.transform(df3).show(truncate=False)

+---+---------------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------+
|ID |oracion                                      |palabras                                             |ngrama                                                                                               |
+---+---------------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------+
|0  |Hola esta prueba es en el año 2021           |[hola, esta, prueba, es, en, el, año]                |[hola esta prueba, esta prueba es, prueba es en, es en el, en el año]                                |
|1  |Spark permite clasificación, regresión, etc. |[spark, permite, clasificación, regresión, etc]      |[spark permite clasificación, permite clasificación regresión, 